In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from datetime import datetime

df = pd.read_csv("decompose_by_interest.csv")
dam = pd.read_csv('2010~2023_dam.csv')
seoul = pd.read_csv('2010~2023_seoul_precipitation.csv')
kangwon = pd.read_csv('2010~2023_kangwon_precipitation.csv')

In [34]:
df.drop('rf',  axis=1, inplace=True)
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

print(df.shape)
df

(1471716, 22)


,datetime,fw,wl,wlobscd,ymdhm,observatory,interest,attention,caution,serious,...,date,year,month,day,hour,warning level,trend,seasonal,anomalies,residual
0,2010-01-01 00:00:00,6.08,1.78,1018655,2010010100,Daegok Bridge,3.8,5.5,7.0,8.5,...,2010-01-01,2010,1,1,0,normal,1.27668,-0.044875,False,-0.000411
1,2010-01-01 01:00:00,6.08,1.78,1018655,2010010101,Daegok Bridge,3.8,5.5,7.0,8.5,...,2010-01-01,2010,1,1,1,normal,1.27668,-0.045638,False,-0.000411
2,2010-01-01 02:00:00,6.08,1.78,1018655,2010010102,Daegok Bridge,3.8,5.5,7.0,8.5,...,2010-01-01,2010,1,1,2,normal,1.27668,-0.045633,False,-0.000411
3,2010-01-01 03:00:00,6.08,1.78,1018655,2010010103,Daegok Bridge,3.8,5.5,7.0,8.5,...,2010-01-01,2010,1,1,3,normal,1.27668,-0.049476,False,-0.000411
4,2010-01-01 04:00:00,6.08,1.78,1018655,2010010104,Daegok Bridge,3.8,5.5,7.0,8.5,...,2010-01-01,2010,1,1,4,normal,1.27668,-0.051012,False,-0.000411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471711,2023-12-31 19:00:00,128.90,1.13,1018640,2023123119,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,2023-12-31,2023,12,31,19,normal,1.27668,-0.115449,False,-0.000411
1471712,2023-12-31 20:00:00,124.87,1.13,1018640,2023123120,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,2023-12-31,2023,12,31,20,normal,1.27668,-0.115451,False,-0.000411
1471713,2023-12-31 21:00:00,124.87,1.13,1018640,2023123121,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,2023-12-31,2023,12,31,21,normal,1.27668,-0.108550,False,-0.000411
1471714,2023-12-31 22:00:00,128.90,1.13,1018640,2023123122,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,2023-12-31,2023,12,31,22,normal,1.27668,-0.106243,False,-0.000411


In [35]:
from datetime import datetime

dam = pd.read_csv('2010~2023_dam.csv')
dam['datetime'] = pd.to_datetime(dam['ymdhm'], format='%Y%m%d%H')
dam.drop('ymdhm', axis=1, inplace=True)
dam.fillna(0, inplace=True)
dam['datetime'] = pd.to_datetime(dam['datetime'])

cjd = dam[dam['dam_name'] == 'Chungju Dam'].sort_values(by='datetime').reset_index(drop=True)
# cjd['datetime'] = cjd['datetime'].astype(str)

pdd = dam[dam['dam_name'] == 'Paldang Dam'].sort_values(by='datetime').reset_index(drop=True)
# pdd['datetime'] = pdd['datetime'].astype(str)

In [29]:
# df = pd.merge(df, cjd[['datetime', 'chungju']], on='datetime', how='left')
# df = pd.merge(df, pdd[['datetime', 'paldang']], on='datetime', how='left')

# df.dropna(inplace=True)
# df

In [36]:
# 서울 강수량
seoul['RN'] = seoul['RN'].replace(-9.0, 0)
seoul.rename(columns={'TM':'datetime', 'RN' : 'rf_seoul'}, inplace=True)
seoul.drop(['Unnamed: 0'], axis=1, inplace=True)

kangwon['RN'] = kangwon['RN'].replace(-9.0, 0)
kangwon.rename(columns={'TM':'datetime', 'RN' : 'rf_kangwon'}, inplace=True)
kangwon.drop(['Unnamed: 0'], axis=1, inplace=True)

In [43]:
df_merged = pd.merge(df, seoul, on='datetime', how='left')
df_merged['datetime'] = pd.to_datetime(df_merged['ymdhm'], format='%Y%m%d%H')

start_date = pd.to_datetime('2010-01-01 00:00:00')
end_date = pd.to_datetime('2023-12-31 23:00:00')

cjd = dam[dam['dam_name'] == 'Chungju Dam'].sort_values(by='datetime').reset_index(drop=True)
pdd = dam[dam['dam_name'] == 'Paldang Dam'].sort_values(by='datetime').reset_index(drop=True)

cjd.rename(columns={'tototf':'chungju'}, inplace=True)
pdd.rename(columns={'tototf':'paldang'}, inplace=True)

cjd = cjd[['chungju', 'datetime']]
pdd = pdd[['paldang', 'datetime']]

for hour in range(4, 13, 2):
    timedelta = pd.Timedelta(hours=hour)

    start_date_ago = start_date - timedelta
    end_date_ago = end_date - timedelta

    dam_hour_ago = cjd[(cjd['datetime'] >= start_date_ago) & (cjd['datetime'] <= end_date_ago)].copy()
    dam_hour_ago['datetime'] = dam_hour_ago['datetime'] + timedelta
    dam_hour_ago.rename(columns={'chungju': f'chungju_{hour}hour'}, inplace=True)

    df_merged = pd.merge(df_merged, dam_hour_ago, on='datetime', how='left')

for hour in range(4, 13, 2):
    timedelta = pd.Timedelta(hours=hour)

    start_date_ago = start_date - timedelta
    end_date_ago = end_date - timedelta

    dam_hour_ago = pdd[(pdd['datetime'] >= start_date_ago) & (pdd['datetime'] <= end_date_ago)].copy()
    dam_hour_ago['datetime'] = dam_hour_ago['datetime'] + timedelta
    dam_hour_ago.rename(columns={'paldang': f'paldang_{hour}hour'}, inplace=True)

    df_merged = pd.merge(df_merged, dam_hour_ago, on='datetime', how='left')

df_merged

,datetime,fw,wl,wlobscd,ymdhm,observatory,interest,attention,caution,serious,...,chungju_4hour,chungju_6hour,chungju_8hour,chungju_10hour,chungju_12hour,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour
0,2010-01-01 00:00:00,6.08,1.78,1018655,2010010100,Daegok Bridge,3.8,5.5,7.0,8.5,...,173.00,176.50,170.30,175.00,173.50,0.000,0.000,0.000,0.000,0.000
1,2010-01-01 01:00:00,6.08,1.78,1018655,2010010101,Daegok Bridge,3.8,5.5,7.0,8.5,...,171.60,172.00,172.60,172.20,173.20,0.000,0.000,0.000,0.000,0.000
2,2010-01-01 02:00:00,6.08,1.78,1018655,2010010102,Daegok Bridge,3.8,5.5,7.0,8.5,...,171.50,173.00,176.50,170.30,175.00,0.000,0.000,0.000,0.000,0.000
3,2010-01-01 03:00:00,6.08,1.78,1018655,2010010103,Daegok Bridge,3.8,5.5,7.0,8.5,...,176.00,171.60,172.00,172.60,172.20,0.000,0.000,0.000,0.000,0.000
4,2010-01-01 04:00:00,6.08,1.78,1018655,2010010104,Daegok Bridge,3.8,5.5,7.0,8.5,...,175.90,171.50,173.00,176.50,170.30,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471711,2023-12-31 19:00:00,128.90,1.13,1018640,2023123119,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,170.46,170.50,170.73,163.49,141.46,0.200,11.507,130.316,129.862,0.200
1471712,2023-12-31 20:00:00,124.87,1.13,1018640,2023123120,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,170.35,170.38,170.62,172.63,141.63,127.950,0.200,118.564,139.239,1.120
1471713,2023-12-31 21:00:00,124.87,1.13,1018640,2023123121,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,170.20,170.46,170.50,170.73,163.49,142.493,0.200,11.507,130.316,129.862
1471714,2023-12-31 22:00:00,128.90,1.13,1018640,2023123122,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,169.71,170.35,170.38,170.62,172.63,143.902,127.950,0.200,118.564,139.239


In [44]:
start_date = pd.to_datetime('2010-01-01 00:00:00')
end_date = pd.to_datetime('2023-12-31 23:00:00')
kangwon['datetime'] = pd.to_datetime(kangwon['datetime'])

for hour in range(4, 13, 2):
    timedelta = pd.Timedelta(hours=hour)

    start_date_ago = start_date - timedelta
    end_date_ago = end_date - timedelta

    kangwon_hour_ago = kangwon[(kangwon['datetime'] >= start_date_ago) & (kangwon['datetime'] <= end_date_ago)]
    kangwon_hour_ago['datetime'] = kangwon_hour_ago['datetime'] + timedelta
    kangwon_hour_ago.rename(columns={'rf_kangwon': f'rf_kangwon_{hour}hour'}, inplace=True)

    df_merged['datetime'] = pd.to_datetime(df_merged['datetime'])
    df_merged = pd.merge(df_merged, kangwon_hour_ago, on='datetime', how='left')
df_merged

,datetime,fw,wl,wlobscd,ymdhm,observatory,interest,attention,caution,serious,...,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour,rf_kangwon_4hour,rf_kangwon_6hour,rf_kangwon_8hour,rf_kangwon_10hour,rf_kangwon_12hour
0,2010-01-01 00:00:00,6.08,1.78,1018655,2010010100,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
1,2010-01-01 01:00:00,6.08,1.78,1018655,2010010101,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
2,2010-01-01 02:00:00,6.08,1.78,1018655,2010010102,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
3,2010-01-01 03:00:00,6.08,1.78,1018655,2010010103,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
4,2010-01-01 04:00:00,6.08,1.78,1018655,2010010104,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471711,2023-12-31 19:00:00,128.90,1.13,1018640,2023123119,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,0.200,11.507,130.316,129.862,0.200,0.0,0.0,0.0,0.8,0.0
1471712,2023-12-31 20:00:00,124.87,1.13,1018640,2023123120,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,127.950,0.200,118.564,139.239,1.120,0.0,0.0,2.7,0.0,0.0
1471713,2023-12-31 21:00:00,124.87,1.13,1018640,2023123121,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,142.493,0.200,11.507,130.316,129.862,0.0,0.0,0.0,0.0,0.8
1471714,2023-12-31 22:00:00,128.90,1.13,1018640,2023123122,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,143.902,127.950,0.200,118.564,139.239,0.0,0.0,0.0,2.7,0.0


In [45]:
df_merged.isna().sum()

datetime               0
fw                     0
wl                     0
wlobscd                0
ymdhm                  0
observatory            0
interest               0
attention              0
caution                0
serious                0
latitude               0
longitude              0
date                   0
year                   0
month                  0
day                    0
hour                   0
warning level          0
trend                  0
seasonal               0
anomalies              0
residual               0
rf_seoul             276
chungju_4hour        144
chungju_6hour        216
chungju_8hour        288
chungju_10hour       360
chungju_12hour       432
paldang_4hour        144
paldang_6hour        216
paldang_8hour        288
paldang_10hour       360
paldang_12hour       432
rf_kangwon_4hour     192
rf_kangwon_6hour     192
rf_kangwon_8hour     192
rf_kangwon_10hour    192
rf_kangwon_12hour    192
dtype: int64

In [46]:
df_merged.fillna(0, inplace=True)
df_merged.to_csv('decompose_by_interest_4.csv', encoding='utf-8', index=False)

In [47]:
df_merged

,datetime,fw,wl,wlobscd,ymdhm,observatory,interest,attention,caution,serious,...,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour,rf_kangwon_4hour,rf_kangwon_6hour,rf_kangwon_8hour,rf_kangwon_10hour,rf_kangwon_12hour
0,2010-01-01 00:00:00,6.08,1.78,1018655,2010010100,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
1,2010-01-01 01:00:00,6.08,1.78,1018655,2010010101,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
2,2010-01-01 02:00:00,6.08,1.78,1018655,2010010102,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
3,2010-01-01 03:00:00,6.08,1.78,1018655,2010010103,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
4,2010-01-01 04:00:00,6.08,1.78,1018655,2010010104,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471711,2023-12-31 19:00:00,128.90,1.13,1018640,2023123119,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,0.200,11.507,130.316,129.862,0.200,0.0,0.0,0.0,0.8,0.0
1471712,2023-12-31 20:00:00,124.87,1.13,1018640,2023123120,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,127.950,0.200,118.564,139.239,1.120,0.0,0.0,2.7,0.0,0.0
1471713,2023-12-31 21:00:00,124.87,1.13,1018640,2023123121,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,142.493,0.200,11.507,130.316,129.862,0.0,0.0,0.0,0.0,0.8
1471714,2023-12-31 22:00:00,128.90,1.13,1018640,2023123122,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,143.902,127.950,0.200,118.564,139.239,0.0,0.0,0.0,2.7,0.0
